# Introduction to Bedrock - Fine-Tuning

> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*


In this demo notebook, we demonstrate how to use the Bedrock Python SDK for fine-tuning Bedrock models with your own data. If you have text samples to train and want to adapt the Bedrock models to your domain, you can further fine-tune the Bedrock foundation models by providing your own training datasets. You can upload your datasets to Amazon S3, and provide the S3 bucket path while configuring a Bedrock fine-tuning job. You can also adjust hyper parameters (learning rate, epoch, and batch size) for fine-tuning. After the fine-tuning job of the model with your dataset has completed, you can start using the model for inference in the Bedrock playground application. You can select the fine-tuned model and submit a prompt to the fine-tuned model along with a set of model parameters. The fine-tuned model should generate texts to be more alike your text samples. 

-----------

1. Setup
2. Fine-tuning
3. Testing the fine-tuned model

 Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

---

## 1. Setup

In [2]:
%pip install -U --force-reinstall pandas==2.1.2 datasets==2.14.6

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for pandas==2.1.2 from https://files.pythonhosted.org/packages/02/52/815f643ed3afb3365354548b3c8b557dbf926a65c40ad5b6d9e455147c7e/pandas-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for numpy<2,>=1.22.4 from https://files.pythonhosted.org/packages/64/41/284783f1014685201e447ea976e85fed0e351f5debbaf3ee6d7645521f1d/numpy-1.26.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 243.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 221.2 MB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 

#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [6]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [7]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

In [8]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [9]:
for model in bedrock.list_foundation_models(byProvider="amazon", byCustomizationType="FINE_TUNING")["modelSummaries"]:
    print("-----\n" + "modelArn: " + model["modelArn"] + "\nmodelId: " + model["modelId"] + "\nmodelName: " + model["modelName"] + "\ncustomizationsSupported: " + ','.join(model["customizationsSupported"]))

-----
modelArn: arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-lite-v1:0:4k
modelId: amazon.titan-text-lite-v1:0:4k
modelName: Titan Text G1 - Lite
customizationsSupported: FINE_TUNING
-----
modelArn: arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-express-v1:0:8k
modelId: amazon.titan-text-express-v1:0:8k
modelName: Titan Text G1 - Express
customizationsSupported: FINE_TUNING


### Invoke Model before Fine-Training

In [10]:
base_model_id = "amazon.titan-text-express-v1"

### Convert the dataset into jsonlines format

In [11]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="gaia_questions_answers.csv")

In [12]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 110
    })
})


In [13]:
def wrap_instruction_fn(example):
    prompt = 'Answer the following question:\n\n'
    end_prompt = '\n\nAnswer: '
    example["instruction"] = prompt + example["question"] + end_prompt
    return example

In [14]:
dataset['train']\
  .filter(lambda example: example['question'] and example['answer'])\
  .select(range(10))\
  .select_columns(['question', 'answer'])\
  .map(wrap_instruction_fn)\
  .rename_column('instruction', 'prompt')\
  .rename_column('answer', 'completion')\
  .remove_columns(['question'])\
  .to_json('./train-question-answer.jsonl', index=False)

# dataset['validation']\
#   .filter(lambda example: example['question'])\
#   .select_columns(['question', 'answer'])\
#   .map(wrap_instruction_fn)\
#   .rename_column('instruction', 'input')\
#   .rename_column('answer', 'output')\
#   .to_json('./validation-summarization.jsonl', index=False)

# dataset['test']\
#   .filter(lambda example: example['question'])\
#   .select_columns(['question', 'answer'])\
#   .map(wrap_instruction_fn)\
#   .rename_column('instruction', 'input')\
#   .rename_column('answer', 'output')\
#   .to_json('./test-summarization.jsonl', index=False)

#  .remove_columns(['Unnamed: 0.1', 'Unnamed: 0', 'question_id', 'title', 'answer_id', 'expertreviewed', 'upvotedcount', 'tags'])\

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

3517

In [15]:
import pandas as pd
df = pd.read_json("./train-question-answer.jsonl", lines=True)
df

,completion,prompt
0,"Intelligent search, automated customer-support...",Answer the following question:\n\nWhat are som...
1,Foundation models are large and complex neural...,Answer the following question:\n\nHow are foun...
2,"The generative AI project life cycle, though n...",Answer the following question:\n\nCan you desc...
3,AWS offers a range of frameworks and infrastru...,Answer the following question:\n\nWhat makes A...
4,"AWS offers increased flexibility, choice, ente...",Answer the following question:\n\nHow does Gen...
5,Building generative AI applications on AWS inv...,Answer the following question:\n\nWhat are the...
6,The project life cycle includes stages like de...,Answer the following question:\n\nHow do proje...
7,AWS's provision of multiple components for bui...,Answer the following question:\n\nHow does AWS...
8,Foundation models are significant in AWS for g...,Answer the following question:\n\nWhat is the ...
9,Prompts in Generative AI involve text-based in...,Answer the following question:\n\nHow do promp...


In [16]:
data = "./train-question-answer.jsonl"

Read the JSON line file into an object like any normal file

In [17]:
with open(data) as f:
    lines = f.read().splitlines()

#### Load the ‘lines’ object into a pandas Data Frame.

In [18]:
import pandas as pd
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

This intermediate data frame will have only one column with each json object in a row. A sample output is given below.

In [19]:
df_inter['json_element'].apply(json.loads)

0    {'completion': 'Intelligent search, automated ...
1    {'completion': 'Foundation models are large an...
2    {'completion': 'The generative AI project life...
3    {'completion': 'AWS offers a range of framewor...
4    {'completion': 'AWS offers increased flexibili...
5    {'completion': 'Building generative AI applica...
6    {'completion': 'The project life cycle include...
7    {'completion': 'AWS's provision of multiple co...
8    {'completion': 'Foundation models are signific...
9    {'completion': 'Prompts in Generative AI invol...
Name: json_element, dtype: object

Now we will apply json loads function on each row of the ‘json_element’ column. ‘json.loads’ is a decoder function in python which is used to decode a json object into a dictionary. ‘apply’ is a popular function in pandas that takes any function and applies to each row of the pandas dataframe or series.

In [20]:
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

Once decoding is done we will apply the json normalize function to the above result. json normalize will convert any semi-structured json data into a flat table. Here it converts the JSON ‘keys’ to columns and its corresponding values to row elements.

In [21]:
df_final

,completion,prompt
0,"Intelligent search, automated customer-support...",Answer the following question:\n\nWhat are som...
1,Foundation models are large and complex neural...,Answer the following question:\n\nHow are foun...
2,"The generative AI project life cycle, though n...",Answer the following question:\n\nCan you desc...
3,AWS offers a range of frameworks and infrastru...,Answer the following question:\n\nWhat makes A...
4,"AWS offers increased flexibility, choice, ente...",Answer the following question:\n\nHow does Gen...
5,Building generative AI applications on AWS inv...,Answer the following question:\n\nWhat are the...
6,The project life cycle includes stages like de...,Answer the following question:\n\nHow do proje...
7,AWS's provision of multiple components for bui...,Answer the following question:\n\nHow does AWS...
8,Foundation models are significant in AWS for g...,Answer the following question:\n\nWhat is the ...
9,Prompts in Generative AI involve text-based in...,Answer the following question:\n\nHow do promp...


### Uploading data to S3

Next, we need to upload our training dataset to S3:

In [22]:
s3_location = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/train-question-answer.jsonl"
s3_output = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/output"

In [23]:
!aws s3 cp ./train-question-answer.jsonl $s3_location

upload: ./train-question-answer.jsonl to s3://sagemaker-us-east-1-079002598131/bedrock/finetuning/train-question-answer.jsonl


Now we can create the fine-tuning job. 

### ^^ **Note:** Make sure the IAM role you're using has these [IAM policies](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-iam-role.html) attached that allow Amazon Bedrock access to the specified S3 buckets ^^

## 2. Fine-tuning

In [24]:
import time
timestamp = int(time.time())

In [25]:
job_name = "titan-{}".format(timestamp)
job_name

'titan-1701058871'

In [26]:
custom_model_name = "custom-{}".format(job_name)
custom_model_name

'custom-titan-1701058871'

In [28]:
bedrock.create_model_customization_job(
    jobName=job_name,
    customModelName=custom_model_name,
    roleArn=role,
    baseModelIdentifier=base_model_id,
    hyperParameters = {
        "epochCount": "10",
        "batchSize": "1",
        "learningRate": "0.000001",
        "learningRateWarmupSteps": "0"
    },
    trainingDataConfig={"s3Uri": s3_location},
    outputDataConfig={"s3Uri": s3_output},
)

{'ResponseMetadata': {'RequestId': 'a79f03f4-2687-457d-a48a-27f83f46a4b3',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Mon, 27 Nov 2023 04:22:21 GMT',
   'content-type': 'application/json',
   'content-length': '122',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a79f03f4-2687-457d-a48a-27f83f46a4b3'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-east-1:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/g7j3engjpdlp'}

In [29]:
status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]
status

'InProgress'

# Let's periodically check in on the progress.
### The next cell might run for ~40min

In [ ]:
import time

status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]

while status == "InProgress":
    print(status)
    time.sleep(30)
    status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]
    
print(status)

InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed


In [ ]:
completed_job = bedrock.get_model_customization_job(jobIdentifier=job_name)
completed_job

## 3. Testing

Now we can test the fine-tuned model

In [ ]:
bedrock.list_custom_models()

In [ ]:
for job in bedrock.list_model_customization_jobs()["modelCustomizationJobSummaries"]:
    print("-----\n" + "jobArn: " + job["jobArn"] + "\njobName: " + job["jobName"] + "\nstatus: " + job["status"] + "\ncustomModelName: " + job["customModelName"])

## GetCustomModel

In [ ]:
bedrock.get_custom_model(modelIdentifier=custom_model_name)

In [ ]:
custom_model_arn = bedrock.get_custom_model(modelIdentifier=custom_model_name)['modelArn']
custom_model_arn

In [ ]:
base_model_arn = bedrock.get_custom_model(modelIdentifier=custom_model_name)['baseModelArn']
base_model_arn

## **Note:** To invoke custom models, you need to first create a provisioned throughput resource and make requests using that resource.

In [ ]:
provisioned_model_name = "{}-provisioned".format(custom_model_name)
provisioned_model_name

## !! **Note:** SDK currently only supports 1 month and 6 months commitment terms. Go to Bedrock console to manually purchase no commitment term option for testing !!

In [ ]:
# bedrock_admin.create_provisioned_model_throughput(
#     modelUnits = 1,
#     commitmentDuration = "OneMonth", ## Note: SDK is currently missing No Commitment option
#     provisionedModelName = provisioned_model_name,
#     modelId = base_model_arn
# ) 

## ListProvisionedModelThroughputs

In [ ]:
bedrock.list_provisioned_model_throughputs()["provisionedModelSummaries"]

## GetProvisionedModelThroughput

In [ ]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

In [ ]:
#provisioned_model_name = "<YOUR_PROVISIONED_MODEL_NAME>" # e.g. custom-titan-1698257909-provisioned
provisioned_model_name = "custom-titan-1700585925-provisioned"

In [ ]:
provisioned_model_arn = bedrock.get_provisioned_model_throughput(
     provisionedModelId=provisioned_model_name)["provisionedModelArn"]
provisioned_model_arn

In [ ]:
deployment_status = bedrock.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]
deployment_status

## The next cell might run for ~10min

In [ ]:
import time

deployment_status = bedrock.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]

while deployment_status == "Creating":
    
    print(deployment_status)
    time.sleep(30)
    deployment_status = bedrock.get_provisioned_model_throughput(
        provisionedModelId=provisioned_model_name)["status"]  
    
print(deployment_status)

# Qualitative Results with Zero Shot Inference BEFORE and AFTER Fine-Tuning

As with many GenAI applications, a qualitative approach where you ask yourself the question "is my model behaving the way it is supposed to?" is usually a good starting point. In the example below (the same one we started this notebook with), you can see how the fine-tuned model is able to create a reasonable summary of the dialogue compared to the original inability to understand what is being asked of the model.

## Before Fine-Tuning

In [ ]:
base_model_id = "amazon.titan-text-express-v1"

In [ ]:
prompt = "Answer the following question:\n\nHow can prompt engineering go wrong?\n\nAnswer: "

In [ ]:
body = {
    "inputText": prompt,
    "textGenerationConfig": {
        "temperature": 0.5,
        "topP": 0.9,
        "maxTokenCount": 512,
        "stopSequences": []
    }
}

response = bedrock_runtime.invoke_model(
    modelId=base_model_id, 
    body=json.dumps(body)
)

response_body = response["body"].read().decode('utf8')
print(json.loads(response_body)["generation"])

In [ ]:
# response = bedrock_runtime.invoke_model(
#     # modelId needs to be Provisioned Throughput Model ARN
#     modelId=base_model_id,
#     body="""
# {
#   "inputText": "Question: How can I create IPv6 mount targets for EFS file systems in an AWS VPC?\\n\\nAnswer: ",
#   "textGenerationConfig":{
#     "maxTokenCount": 1000, 
#     "stopSequences": [],
#     "temperature": 0.1,
#     "topP": 0.9
#   }
# }
# """
# )

# response_body = response["body"].read().decode('utf8')
# print(response_body)

# print(json.loads(response_body)["results"][0]["outputText"])

## After Fine-Tuning

In [ ]:
body = {
    "inputText": prompt,
    "textGenerationConfig": {
        "temperature": 0.5,
        "topP": 0.9,
        "maxTokenCount": 512,
        "stopSequences": []
    }
}

response = bedrock_runtime.invoke_model(
    modelId=provisioned_model_arn, 
    body=json.dumps(body)
)

response_body = response["body"].read().decode('utf8')
print(json.loads(response_body)["generation"])

In [ ]:
# response = bedrock_runtime.invoke_model(
#     # modelId needs to be Provisioned Throughput Model ARN
#     modelId=provisioned_model_arn,
#     body="""
# {
#   "inputText": "Question: What is the workaround to creating IPv6 mount targets for EFS file systems in an AWS VPC?\\n\\nAnswer: ",
#   "textGenerationConfig":{
#     "maxTokenCount": 500, 
#     "stopSequences": [],
#     "temperature": 0.1,
#     "topP": 0.9
#   }
# }
# """
# )

# response_body = response["body"].read().decode('utf8')
# print(response_body)

# print(json.loads(response_body)["results"][0]["outputText"])

## Delete Provisioned Throughput

When you're done testing, you can delete Provisioned Throughput to stop charges

In [ ]:
# bedrock_admin.delete_provisioned_model_throughput(
#     provisionedModelId = provisioned_model_name
# )